In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import generator

In [ ]:
shape = (100, 100, 100)
porosity = np.array([0.5])
blobiness = np.array([1, 2, 5])
noise_method = 'poisson'
noise = np.array([1e2, 1e3, 1e5])
num_of_angles = 180
tag = 'train'


def generate_phantoms(preview=True):

    for p in porosity:
        for b in blobiness:
            for n in noise:
                generator.create_phantom_and_process(shape, p, b, n, num_of_angles, tag, preview=preview, noise_method=noise_method)

                
generate_phantoms()

In [ ]:
import data_manager as dm
data_info = dm.show_data_info()
data_info

In [ ]:
import pixel_counter as pc
import matplotlib.pyplot as plt

tag = 'train'
dim = 3

im_lenght = np.prod(shape)
coeff = 100
sample_lenght = (im_lenght/coeff).astype(np.int)
indices = (np.random.rand(sample_lenght) * im_lenght).astype(np.int)


def scatter_plot(x, y, colors, title):
    plt.figure(figsize=(10, 10))
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.scatter(x, y, marker='.', c=colors)
    plt.title(title)

    
def get_params_from_data_info(data_info, index):

    porosity = data_info['porosity'][index[0]]
    blobiness = data_info['blobiness'][index[0]]
    noise = data_info['noise'][index[0]]

    return porosity, blobiness, noise

    
def count_npa_vs_pp():

    for index, data_id in np.ndenumerate(data_info['id_indx']):
        
        porosity, blobiness, noise = get_params_from_data_info(data_info, index)
        title = f'porosity { porosity }, blobiness { blobiness }, noise { noise }'
        
        pp, npa, op = pc.count_neighbor_average_array_and_save(dim, data_id, tag)

        # pp — proc_phantom
        # npa — neighbor_pixel_average
        # op — orig_phantom

        pp_part = np.take(pp, indices)
        npa_part = np.take(npa, indices)
        op_part = np.take(op, indices)
        colors = ['red' if el else 'blue' for el in op_part]
        scatter_plot(pp_part, npa_part, colors, title)
        
        
count_npa_vs_pp()

In [ ]:
tag = 'test'
generate_phantoms(preview=False)

In [ ]:
data_info = dm.show_data_info()
data_info

In [ ]:
from sklearn import preprocessing

dim = 3
tag = 'train'


def train_model():

    for index, data_id in np.ndenumerate(data_info['id_indx']):
        
        porosity, blobiness, noise = get_params_from_data_info(data_info, index)
        title = f'porosity { porosity }, blobiness { blobiness }, noise { noise }'
        
        pixels_df = dm.get_data(dimension=dim, id_indx=data_id, what_to_return='csv', tag=tag)
        X_train = np.asarray(pixels_df[['neighbor_average', 'proc_phantom_pixel_values']])
        Y_train = np.asarray(pixels_df['pixel_real_value'])
        scaler = preprocessing.StandardScaler()
        scaler.fit(X_train)
        scaler.transform(X_train)
        
        print(title)
        print(f'mean: { scaler.mean_ }, var: { scaler.var_ }, samples_seen: { scaler.n_samples_seen_ }')
        

# train_model()